In [ ]:
WITH InitialSales AS (
    SELECT  
        a.cIDCompany Company_DwID,
        a.cIDInvoice Invoice_DwID,
        a.cSerie Serie,
        a.cForm Form,
        a.cIDCustomer Customer_DwID,
        a.cIDBranchInvoice BranchDwID,
        a.cIDTerritory Territory_DwID,
        a.dEmission Emission,
        a.xChannelDistribution ChannelDistribution,
        a.xReturnReason InvoiceCancelReason,
        b.cidOrder Order_DwID,
        a.xInvoiceType InvoiceType,
        e.cIDProduct Product_DwID,
        e.nAmount Amount,
        e.nUnitValue UnitValue,
        e.nTotalValue TotalValue,
        e.MC1LastUpdate,
        e.nFreegoods Freegoods,
        g.nSeq Seq,
        g.nValue Value,
        g.cIDPricing Pricing_DwID,
        g.cIDStep Step_DwID,
        f.xIDReasonReturnStock ReasonReturnStock_DwID,
        h.lCalculated Calculated,
        h.cIDPricingFormationRule PricingFormationRule_DwID
    FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_Invoice a
    --################# order invoice #########################
    INNER JOIN (SELECT *
                FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_OrderInvoice 
               ) b 
    ON a.cIDCompany = b.cIDCompany
    AND a.cIDInvoice = b.cIDInvoice
    AND a.cForm = b.cform
    AND a.cSerie = b.cSerie
    AND a.cIDBranchInvoice = b.cIDBranchInvoice
    AND a.mc1Enabled = 1
    AND b.mc1enabled = 1
    --################## invoice item #####################
    INNER JOIN (SELECT *
                FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_InvoiceItem
               ) e 
    ON e.cIDCompany = a.cIDCompany 
    AND e.cIDInvoice = a.cIDInvoice 
    AND e.cForm = a.cform
    AND e.cSerie = a.cSerie
    AND e.cIDBranchInvoice = a.cIDBranchInvoice 
    AND e.mc1enabled = 1
    --################ invoice item detail #######################
    INNER JOIN (SELECT *
                FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_InvoiceItemDetail 
               ) g
    ON g.cIDCompany = e.cIDCompany
    AND g.cIDInvoice = e.cIDInvoice
    AND g.cForm = e.cform
    AND g.cSerie = e.cSerie
    AND g.cIDBranchInvoice = e.cIDBranchInvoice
    AND g.nSeq = e.nSeq --igualando pasos 
    AND g.cidProduct = e.cidProduct --igualando los productos de factura y order
    AND g.mc1enabled = 1
    AND g.cIDStep IN ('00100','00130','00160','00200','00250','00300','00400','00450','00500','00600','00700','00800','00900','01400','00150') --steps para llegar a la venta neta 
    --################### MC1_MC_OrderExt ########################
    INNER JOIN (SELECT *
                FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_OrderExt
               ) f 
    ON f.cIDOrder = b.cIDOrder
    AND f.cIDCompany = b.cIDCompany
    AND f.mc1enabled = 1
    --#################### REQUEST EMITES ######################
    INNER JOIN (SELECT * 
                FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_InvoiceRequestEmites
               ) m
    ON a.cIDCompany = m.cIDCompany
    AND a.cIDInvoice = m.cIDInvoice
    AND a.cForm = m.cform
    AND a.cSerie = m.cSerie
    AND a.cIDBranchInvoice = m.cIDBranchInvoice
    AND m.mc1enabled = 1
    AND UPPER(m.xInvoiceRequestStatus) = 'SUCCEEDED' 
    --################### PRICING_RULE ###################
    INNER JOIN (SELECT *
                FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_PricingFormationRule
               ) h
    ON g.cIDStep = h.cIDStep
    AND g.cIDPricing = h.cIDPricing
    AND g.cIDCompany = h.cIDCompany
    AND g.mc1enabled = 1
    AND h.mc1enabled = 1
    AND UPPER(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED' --se repite el paso 00100 y parece aumentar el valor 
    WHERE a.xInvoiceType NOT IN ('WTM002','WTM005','ZOGRD','ZOGR') --Ordertype no buscados a fin de ver os nuevos
    AND FORMAT(a.dEmission , 'yyyy-MM-dd') BETWEEN '{inicio}' AND '{fin}' --informacion de un dia  
    ORDER BY a.cIDCompany, a.cIDInvoice, a.cSerie, a.cForm, a.cIDCustomer, a.cIDBranchInvoice, a.cIDTerritory, a.dEmission, a.xChannelDistribution, a.xReturnReason, b.cidOrder, a.xInvoiceType, e.cIDProduct, e.nAmount, e.nUnitValue, e.nTotalValue, e.MC1LastUpdate, e.nFreegoods, g.nSeq, g.nValue, g.cIDPricing, g.cIDStep, f.xIDReasonReturnStock, h.lCalculated, h.cIDPricingFormationRule
),
ValidOrders AS (
    SELECT 
        a.cIDCompany,
        a.cIDBranchInvoice,
        a.cSerie,
        a.cIDInvoice,
        a.cForm,
        a.cIDProduct,
        FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm') AS MC1LastUpdate,
        c.nfreegoods,
        a.nValue AS Value2
    --,sum(a.Value) Value2 ,COUNT(*) no_registros
    FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_InvoiceItemDetail a
    INNER JOIN (
        SELECT * 
        FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_Invoice 
        WHERE MC1Enabled = 1
        AND FORMAT(DEmission, 'yyyy-MM-dd') BETWEEN '{inicio}' AND '{fin}' --informacion de un dia  
        AND xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR')
    ) b
    ON a.cIDBranchInvoice = b.cIDBranchInvoice
    AND a.cSerie = b.cSerie
    AND a.cIDInvoice = b.cIDInvoice
    AND a.cForm = b.cForm
    INNER JOIN (
        SELECT * 
        FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_Invoiceitem
        WHERE MC1Enabled = 1
    ) c
    ON a.cIDBranchInvoice = c.cIDBranchInvoice
    AND a.cSerie = c.cSerie
    AND a.cIDInvoice = c.cIDInvoice
    AND a.cForm = c.cForm
    AND a.cIDProduct = c.cIDProduct
    AND a.nSeq = c.nSeq
    WHERE a.cIDStep = '00800'
    AND a.MC1Enabled = 1
    GROUP BY 
        a.cIDCompany,
        a.cIDBranchInvoice,
        a.cSerie,
        a.cIDInvoice,
        a.cForm,
        a.cIDProduct,
        FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm'),
        c.nfreegoods,
        a.nValue
),
ValidSales AS (
    SELECT 
        a.* 
    FROM InitialSales a
    INNER JOIN ValidOrders b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct AND
        a.BranchDwID = b.cIDBranchInvoice
        AND b.cIDOrder is not null
    --WHERE b.cIDOrder is not null
),
FirstDeduplicates AS(
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.BranchDwID, a.ChannelDistribution, 
            a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.FreeGoods, a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType,
            a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID ORDER BY a.mc1LastUpdate DESC, a.ReasonReturnStock_DwID DESC) as rn
    FROM ValidSales a
    ) sub
    WHERE rn = 1
),
Transformations AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN a.Value ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00400' THEN a.Value ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00200' THEN a.Value ELSE 0 END AS Descuento_ML,
        CASE WHEN a.Step_DwID = '00150' THEN a.Value ELSE 0 END AS Escalafc_ML,
        CASE 
            WHEN a.InvoiceType ='ZBON' AND a.Step_DwID = '00800' THEN a.Value
            WHEN a.Step_DwID = '00600' AND a.Freegoods = 0 THEN a.Value
            WHEN a.Step_DwID = '00100' AND a.Freegoods = 1 THEN a.Value 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '00900' THEN a.Value ELSE 0 END AS Impuesto1,
        0 as Impuesto2,
        CASE WHEN a.Step_DwID = '01400' THEN a.Value ELSE 0 END AS Venta_Impuestos
    FROM FirstDeduplicates a
),
GroupedSales AS (
    SELECT
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM Transformations
    GROUP BY
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue
),
GroupesSales2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM GroupedSales
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Territory_DwID,
        UnitValue
),
InvoiceItemDetailJoin AS (
    SELECT 
        a.cIDCompany Company_DwID,
        a.cIDBranchInvoice BranchInvoice_DwID,
        a.cSerie Serie,
        a.cIDInvoice Invoice_DwID,
        a.cForm Form,
        a.cIDProduct Product_DwID,
        FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm') AS MC1LastUpdate,
        c.nfreegoods FreeGoods,
        a.nValue AS Value2
    --,sum(a.Value) Value2 ,COUNT(*) no_registros
    FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_InvoiceItemDetail a
    INNER JOIN (
        SELECT * 
        FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_Invoice 
        WHERE MC1Enabled = 1
        AND FORMAT(DEmission, 'yyyy-MM-dd') BETWEEN '{inicio}' AND '{fin}' --informacion de un dia  
        AND xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR')
    ) b
    ON a.cIDBranchInvoice = b.cIDBranchInvoice
    AND a.cSerie = b.cSerie
    AND a.cIDInvoice = b.cIDInvoice
    AND a.cForm = b.cForm
    INNER JOIN (
        SELECT * 
        FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_Invoiceitem
        WHERE MC1Enabled = 1
    ) c
    ON a.cIDBranchInvoice = c.cIDBranchInvoice
    AND a.cSerie = c.cSerie
    AND a.cIDInvoice = c.cIDInvoice
    AND a.cForm = c.cForm
    AND a.cIDProduct = c.cIDProduct
    AND a.nSeq = c.nSeq
    WHERE a.cIDStep = '00800'
    AND a.MC1Enabled = 1
    GROUP BY 
        a.cIDCompany,
        a.cIDBranchInvoice,
        a.cSerie,
        a.cIDInvoice,
        a.cForm,
        a.cIDProduct,
        FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm'),
        c.nfreegoods,
        a.nValue
),
InvoiceItemDetailDeduplicated AS (
    SELECT distinct
        a.*
    FROM InvoiceItemDetailJoin a
),
InvoiceDetailGenerated AS (
    SELECT 
        Company_DwID,
        BranchInvoice_DwID,
        Serie,
        Invoice_DwID,
        Form,
        Product_DwID,
        MC1LastUpdate,
        FreeGoods,
        CASE WHEN FreeGoods = 1 THEN 0 ELSE Value2 END AS Value2
    FROM InvoiceItemDetailDeduplicated
),
InvoiceDetailGrouped AS (
    SELECT
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie,
        SUM(Value2) AS facdet
    FROM InvoiceDetailGenerated
    GROUP BY 
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie
),
FinalJoin AS (
    SELECT 
        a.*,
        b.facdet
    FROM GroupesSales2 a
    INNER JOIN InvoiceDetailGrouped b ON
        a.Company_DwID = b.Company_DwID AND
        a.BranchDwID = b.BranchInvoice_DwID AND
        a.Product_DwID = b.Product_DwID AND
        a.Serie = b.Serie AND
        a.Form = b.Form AND
        a.Invoice_DwID = b.Invoice_DwID
),
AddColumns AS (
    SELECT 
        a.BranchDwID codigo_agencia,
        a.ChannelDistribution,
        CASE WHEN a.Company_DwID='UY' THEN 'URUGUAY' ELSE 'REV' END as pais,
        a.Customer_DwID codigo_cliente,
        CONVERT(VARCHAR(10), CAST(a.Emission AS DATE), 120) fecha,
        a.Form,
        a.Invoice_DwID,
        a.InvoiceCancelReason,
        a.InvoiceType AG19ID,
        a.Order_DwID,
        a.Product_DwID codigo_producto,
        a.ReasonReturnStock_DwID,
        a.Serie,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') as codigo_ruta,
        a.UnitValue,
        a.Valor_Bruto,
        a.Facdsl_ML,
        a.Promocion_ML,
        a.Descuento_ML,
        a.Amount,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.Escalafc_ML,
        CONCAT(a.Serie,'-',a.Invoice_DwID) as numero,
        a.ChannelDistribution as cod_linea_ruta,
        'FALSE' as FACCON,
        0 as FACDSL,
        null as dif_precio,
        CASE
            WHEN a.InvoiceType = 'ZBON' THEN 0
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN 0 - facdet
            ELSE facdet
        END AS facdet,
        CASE 
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN a.ReasonReturnStock_DwID
            WHEN NOT (a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX') THEN a.InvoiceCancelReason
            ELSE 'REVISAR'
        END AS Id_Motivo
    FROM FinalJoin a
),
Subchannel AS (
    SELECT *
    FROM (
        SELECT 
            a.*,
            b.cidbranchinvoice,
            ROW_NUMBER() OVER (PARTITION BY b.cidbranchinvoice, a.cIDCustomer ORDER BY a.mc1Enabled DESC, a.mc1LastUpdate DESC) AS rn
        FROM CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_CustomerDivisionSectorExt a
        LEFT JOIN CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_CustomerDivisionSector b 
        ON a.cidcustomer = b.cidcustomer
        WHERE b.cidbranchinvoice IS NOT NULL
    ) sub
    WHERE rn = 1
),
JoinWithSubchannel AS (
    SELECT
        a.*,
        b.xSubChannelDistribution
    FROM AddColumns a
    LEFT JOIN  Subchannel b
    ON a.codigo_cliente = b.cIDCustomer
    AND a.codigo_agencia = b.cidbranchinvoice
),
ModifyColumns AS (
    SELECT 
        a.codigo_agencia,
        a.codigo_cliente CLICOD,
        CONVERT(VARCHAR(10), a.fecha, 120) Fecha_vta,
        a.Invoice_DwID nof_factura,
        a.InvoiceCancelReason,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.UnitValue precio,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Facdsl_ML
            ELSE a.Facdsl_ML END AS FACDSL$ ,
         CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Promocion_ML
            ELSE a.Promocion_ML END AS promocion,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Descuento_ML
            ELSE a.Descuento_ML END AS descuento,
        a.Amount cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        CASE
            WHEN RTRIM(LTRIM(a.AG19ID)) = 'ZBON' OR RTRIM(LTRIM(a.AG19ID)) = 'ZOREX' OR RTRIM(LTRIM(a.AG19ID)) = 'ZREX'
            THEN 0-a.Escalafc_ML
            ELSE a.Escalafc_ML END AS Escalafc_ML,
        a.numero,
        a.FACCON,
        a.FACDSL,
        a.facdet FACDET,
        CASE  
    		WHEN a.cod_linea_ruta <> a.xSubChannelDistribution AND a.xSubChannelDistribution IS NOT NULL THEN a.xSubChannelDistribution ELSE a.cod_linea_ruta 
		END AS cod_linea_ruta,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason
            WHEN upper(a.Id_Motivo)='WTM002' and a.InvoiceCancelReason <> ''  THEN a.InvoiceCancelReason 
            ELSE a.Id_Motivo END as id_motivoventa,
        CASE 
            WHEN a.Id_Motivo is null AND a.InvoiceCancelReason <> ''  THEN CONCAT(a.Id_Motivo, '-',a.InvoiceCancelReason)
            WHEN upper(a.Id_Motivo) <> 'WTM001'  THEN CONCAT(a.Id_Motivo, '-',a.InvoiceCancelReason)
            ELSE CONCAT(a.ReasonReturnStock_DwID, '-', a.InvoiceCancelReason) END as Id_Motivo
    FROM JoinWithSubchannel a 
),
ReturnReason AS (
    SELECT 
        CASE
            WHEN cDescription LIKE '%[A-Za-z]%' THEN UPPER(cDescription)
            ELSE cDescription
        END as [Description],
        cDomainType, 
        cIDDomainType, 
        lEnabled
    from CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_GeneralDescription
    Where cDomainType = 'xReturnReason'
    AND lEnabled = 1
    AND UPPER(cDescription) <> 'SOBRESTOCK EN TIENDA'
),
Motivos AS(
    SELECT 
        CASE
            WHEN cDescription LIKE '%[A-Za-z]%' THEN UPPER(cDescription)
            ELSE cDescription
        END as [Description],
        cDomainType, 
        cIDDomainType, 
        lEnabled
    from CBRTM3XPOLCD22_LASC1.GRO_BU_RTM.VW_MC1_GeneralDescription
    Where cDomainType = 'xIDReasonReturnStock'
    AND lEnabled = 1
    AND UPPER(cDescription) <> 'SOBRESTOCK EN TIENDA'
),
FinalMC1Uruguay as (
    SELECT 
        a.codigo_agencia,
        a.CLICOD,
        a.Fecha_vta,
        a.nof_factura,
        a.AG19ID,
        a.codigo_producto,
        a.codigo_ruta,
        a.precio,
        a.FACDSL$,
        a.promocion,
        a.descuento,
        a.cantidad,
        a.Impuesto1,
        a.Venta_Impuestos,
        a.Impuesto2,
        a.Escalafc_ML,
        a.numero,
        a.FACCON,
        a.FACDSL,
        a.FACDET,
        a.cod_linea_ruta,
        a.Id_Motivo,
        CASE
            WHEN b.Description is null AND c.Description <> ''
            THEN c.Description
            ELSE b.Description
        END AS factura_cliente
    FROM ModifyColumns a
    LEFT JOIN ReturnReason b
    ON a.InvoiceCancelReason = b.cIDDomainType
    LEFT JOIN Motivos c
    ON a.Id_Motivo = c.cIDDomainType
    Where a.codigo_ruta NOT LIKE '%0A%'
    AND a.Fecha_vta  >= '2022-05-16'
)
SELECT * FROM FinalMC1Uruguay